Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [2]:
%%sql
Pragma foreign_key=on;

Done.


[]

In [3]:
%%sql

drop table if exists coaches;

CREATE TABLE coaches
(
id integer primary key,
name varchar(100) 
); 

drop table if exists positions;

CREATE TABLE positions
(
id integer primary key,
name varchar(100) 
); 

drop table if exists playeesarchive;

CREATE TABLE playeesarchive
(
 player_id integer,
 team_id integer,
 stert_date date, 
 end_date date   
); 

drop table if exists teams;

CREATE TABLE teams
(
 id integer primary key,
 name varchar(100), 
 location varchar(100), 
 coach_id integer,
 foreign key (id) references playeesarchive(team_id),     
 foreign key (Coach_id) references coaches(id)   
); 

drop table if exists players;

CREATE TABLE players
(
 id integer primary key,
 name varchar(100), 
 age varchar(100), 
 position_id integer,
 team_id integer,  
 start_date date,
 foreign key (id) references playeesarchive(player_id),     
 foreign key (position_id) references positions(id),     
 foreign key (team_id) references teams(id)
); 




Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [4]:
%%sql
insert into positions values(1, "goalkeeper");
insert into positions values(2, "defender");
insert into positions values(3, "midfielder");
insert into positions values(4, "forward");

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [5]:
import sqlite3 # Импортируем библиотеку, соответствующую типу нашей базы данных SQLite
db=sqlite3.connect('chinook.db') # Создаем соединение с нашей базой данных

In [6]:

def insert_trainer(name):
# Создаем курсор - это специальный объект который делает запросы и получает их результаты
    cur = db.cursor()
    cur.execute( ''' SELECT COALESCE(MAX(id)+1,1) FROM Coaches ''') # запросим адрес послепоследней строки
    ID = cur.fetchone()[0] # запишем в ID
    t = bool(1)
    try:
        for i in range(len(name)):
            name[i]>='1' and name[i]<='9'
    except TypeError:
        print('имя должно быть строкой str')
        return -1        
   
    cur.execute(''' INSERT INTO coaches(id, name) values(?,?) ''', (ID, name))
    db.commit() # Если мы не просто читаем, но и вносим изменения в базу данных - необходимо сохранить транзакцию
# Не забываем закрыть соединение с базой данных
#db.close()

In [7]:
insert_trainer("Виктор Иванов")
insert_trainer("Зинедин Зидан")

In [8]:
insert_trainer(123)

имя должно быть строкой str


-1

In [9]:
%%sql
SELECT * FROM Coaches

Done.


id,name
1,Виктор Иванов
2,Зинедин Зидан


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [10]:
import sqlite3
db=sqlite3.connect('chinook.db')
def insert_teams(name, location, coach_name):
    cur = db.cursor()
    cur.execute( ''' SELECT COALESCE(MAX(id)+1, 1) FROM Teams ''')
    i = cur.fetchone()[0]
    cur.execute('''SELECT coaches.id FROM coaches WHERE coaches.name=? ''', (coach_name,)) # запросим id с нужным именем
    coach_id = cur.fetchone()[0]
    cur.execute(''' INSERT INTO teams(id, name, location, coach_id) VALUES(?,?,?,?)''', (i, name, location, coach_id,))
    db.commit()

In [11]:
insert_teams('Локомотив','Москва','Виктор Иванов');
insert_teams('Зенит','Санкт-Петербург','Зинедин Зидан');

In [12]:
%%sql
SELECT * FROM Teams

Done.


id,name,location,coach_id
1,Локомотив,Москва,1
2,Зенит,Санкт-Петербург,2


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [13]:
insert_teams('Бешикташ','Москва','Виктор Иванов');

In [14]:
insert_trainer("Турк Укроп")

In [17]:
import sqlite3
db=sqlite3.connect('chinook.db')
def update_team(name, location=None, coach_name=None): # начальные значения места и имени нулевые
    cur = db.cursor()
    if(location == None and coach_name == None):  # если информации нет, то ничего и не обновляем
        return;
    if(location == None and coach_name != None):  # если есть имя тренера
        cur.execute('''select Coaches.id from Coaches where Coaches.name=? ''', (coach_name,)) #из таблицы тренеров получим нужный 
        c_id = cur.fetchone()[0]                                                           # id тренера
        cur.execute('''Update Teams set coach_id = ? where name = ?''', (c_id, name)) # вызовем update функцию, которая обновит
    if(location != None and coach_name == None):                                      # присвоит индексу новое имя
        cur.execute('''Update Teams set location = ? where name = ?''', (location, name)) #  прямая замена места
    if(location != None and coach_name != None):
        cur.execute('''SELECT Coaches.id FROM Coaches WHERE Coaches.Name=? ''', (coach_name,))
        c_id = cur.fetchone()[0]
        cur.execute('''Update Teams set coach_id = ? where name = ?''', (c_id, name,))
        cur.execute('''Update Teams set location = ? where name = ?''', (location, name,))
    db.commit()

In [15]:
%%sql
SELECT * FROM Teams

Done.


id,name,location,coach_id
1,Локомотив,Москва,1
2,Зенит,Санкт-Петербург,2
3,Бешикташ,Москва,1


In [18]:
update_team("Бешикташ", "Станбул", "Турк Укроп")

In [19]:
update_team("Локомотив", "Барселона")

In [20]:
%%sql
SELECT * FROM Teams

Done.


id,name,location,coach_id
1,Локомотив,Барселона,1
2,Зенит,Санкт-Петербург,2
3,Бешикташ,Станбул,3


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [21]:
import sqlite3
db=sqlite3.connect('chinook.db')
def insert_players(name, age, position, team, start_date=None): 
    cur = db.cursor() 
    cur.execute( '''select COALESCE(MAX(id)+1, 1) FROM players''') 
    try: 
        ID = cur.fetchone()[0] 
        cur.execute( ''' select id from positions where name=? ''', (position,)); 
        position_id = cur.fetchone()[0]; 
        cur.execute( ''' select id from teams where name=? ''', (team,)); 
        team_id = cur.fetchone()[0];
        cur.execute( ''' select id from positions ''');
        pos = cur.fetchall();
        if start_date==None: 
            cur.execute( ''' select DATE (CURRENT_DATE) ''' ) 
            start_date=cur.fetchone()[0]; 
        if ((position_id,) in pos and (age>10) and (age < 50)): 
            cur.execute("""insert into players(id,name, age,position_id,team_id, start_date ) VALUES(?,?,?,?,?,?)""", (ID,name,age, position_id,team_id, start_date)) 
        else: 
            print("Error1") 
        db.commit() 
    except TypeError: 
        print("Error2")

In [22]:
insert_players("Ашот Ибрагимович", 31, "goalkeeper", "Зенит")


In [23]:
%%sql
SELECT * FROM PLAYERS

Done.


id,name,age,position_id,team_id,start_date
1,Ашот Ибрагимович,31,1,2,2018-05-26


In [24]:
insert_players("Александр Овечкин", 20, "forward", "Локомотив")

In [25]:
insert_players("Антон Макаревич", 25, "defender", "Локомотив", 1995-11-27)

In [26]:
%%sql
SELECT * FROM PLAYERS

Done.


id,name,age,position_id,team_id,start_date
1,Ашот Ибрагимович,31,1,2,2018-05-26
2,Александр Овечкин,20,4,1,2018-05-26
3,Антон Макаревич,25,2,1,1957


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [35]:
import sqlite3
db=sqlite3.connect('chinook.db')
def update_players(name, new_team, date): 
    cur = db.cursor() 
    cur.execute( '''select DATE(?, '-1 day')''',(date,)) 
    try: 
        d=cur.fetchone()[0] 
        cur.execute( '''select team_id from players where name=?''',(name,)) 
        t= cur.fetchone()[0] 
        cur.execute( '''select id from players where name=?''',(name,)) 
        ID= cur.fetchone()[0] 
        cur.execute('''Update playeesarchive set end_date = ? where player_id = ? and team_id =?''', (d, ID, t,)) 
        db.commit()
        cur.execute( '''select id from teams where name=?''',(new_team,)) 
        tn= cur.fetchone()[0] 
        cur.execute ('''Update players set team_id = ?, start_date=? where name = ?''', (tn, date, name)) 
        cur.execute(''' insert into playeesarchive (player_id, team_id, stert_date, end_date ) VALUES(?,?,?,?)''', (ID,tn,date,'')) 
        db.commit() 
    except TypeError: 
        print("Error") 
        db.rollback()

In [43]:
update_players("Ашот Ибрагимович", "Локомотив", "1995-11-27")

In [44]:
%%sql
SELECT * FROM PLAYERS

Done.


id,name,age,position_id,team_id,start_date
1,Ашот Ибрагимович,31,1,1,1995-11-27
2,Александр Овечкин,20,4,2,1999-01-01
3,Антон Макаревич,25,2,1,1957


In [40]:
update_players("Александр Овечкин", "Зенит", "1999-01-01")

In [41]:
%%sql
SELECT * FROM PLAYERS

Done.


id,name,age,position_id,team_id,start_date
1,Ашот Ибрагимович,31,1,1,1957
2,Александр Овечкин,20,4,2,1999-01-01
3,Антон Макаревич,25,2,1,1957


In [45]:
%%sql
SELECT * FROM playeesarchive

Done.


player_id,team_id,stert_date,end_date
1,1,1957,1995-11-26
2,2,1999-01-01,
1,1,1995-11-27,
